In [1]:
# Import python packages
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process as fwprocess
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [2]:
# Can uncomment to install R packages as needed
#utils = importr('utils')
#utils.install_packages('nflreadr')
#utils.install_packages('ffscrapr')

In [3]:
# Import R packages
nflreadr = importr('nflreadr')
ffscrapr = importr('ffscrapr')

In [4]:
# Config for the calculation
mfl_id = 60206  #MFL id for league
seasons = [2023,2022,2021]  #Seasons to collect data for
max_week = 17  #Maximum week number to include in calculation
num_off = 7  #Number of offensive players to include in calculation for each game
num_def = 12  #Number of defensive  players to include in calculation for each game
save_label = 'Analytics_Dynasty_League_2024'  #Where to save results

In [5]:
# A mapping from PFR team names to MFL team names

###
team_dic = {
    'ARI' : 'ARI',
    'ATL' : 'ATL',
    'BAL' : 'BAL',
    'BUF' : 'BUF',
    'CAR' : 'CAR',
    'CHI' : 'CHI',
    'CIN' : 'CIN',
    'CLE' : 'CLE',
    'DAL' : 'DAL',
    'DEN' : 'DEN',
    'DET' : 'DET',
    'GB' : 'GBP',
    'HOU' : 'HOU',
    'IND' : 'IND',
    'JAX' : 'JAC',
    'KC' : 'KCC',
    'LA' : 'LAR',
    'LAC' : 'LAC',
    'LV' : 'LVR',
    'MIA' : 'MIA',
    'MIN' : 'MIN',
    'NE' : 'NEP',
    'NO' : 'NOS',
    'NYG' : 'NYG',
    'NYJ' : 'NYJ',
    'PHI' : 'PHI',
    'PIT' : 'PIT',
    'SEA' : 'SEA',
    'SF' : 'SFO',
    'TB' : 'TBB',
    'TEN' : 'TEN',
    'WAS' : 'WAS',
}

In [6]:
# Mappings for positions, both to consider which positions to drop from the analysis and to cleanup PFR positions a bit

###
snap_side_map = {
    'C' : 'DROP',
    'CB' : 'DEF',
    'DB' : 'DEF',
    'DE' : 'DEF',
    'DT' : 'DEF',
    'FB' : 'DROP',  #Change to OFF if want to include FBs
    'FS' : 'DEF',
    'G' : 'DROP',
    'K' : 'DROP',
    'LB' : 'DEF',
    'LS' : 'DROP',
    'NT' : 'DEF',
    'P' : 'DROP',
    'QB' : 'OFF',
    'RB' : 'OFF',
    'SS' : 'DEF',
    'T' : 'DROP',
    'TE' : 'OFF',
    'WR' : 'OFF',
}

snap_position_map = {
    'CB' : 'CB',
    'DB' : 'S',
    'DE' : 'DE',
    'DT' : 'DT',
    'FB' : 'RB',
    'FS' : 'S',
    'LB' : 'LB',
    'NT' : 'DT',
    'QB' : 'QB',
    'RB' : 'RB',
    'SS' : 'S',
    'TE' : 'TE',
    'WR' : 'WR',
}

###
pos_side_map = {
    'CB' : 'DEF',
    'Coach' : 'DROP',
    'DE' : 'DEF',
    'DT' : 'DEF',
    'Def' : 'DROP',
    'LB' : 'DEF',
    'Off' : 'DROP',
    'PK' : 'DROP',
    'PN' : 'DROP',
    'QB' : 'OFF',
    'RB' : 'OFF',
    'S' : 'DEF',
    'ST' : 'DROP',
    'TE' : 'OFF',
    'TMDB' : 'DROP',
    'TMDL' : 'DROP',
    'TMLB' : 'DROP',
    'TMPK' : 'DROP',
    'TMPN' : 'DROP',
    'TMQB' : 'DROP',
    'TMRB' : 'DROP',
    'TMTE' : 'DROP',
    'TMWR' : 'DROP',
    'WR' : 'OFF',
    'XX' : 'DROP'}

In [7]:
# Scrape snap counts from PFR and convert to pandas df
snap_df_r = nflreadr.load_snap_counts(ro.IntVector(seasons))

with (ro.default_converter + pandas2ri.converter).context():
  snap_df = ro.conversion.get_conversion().rpy2py(snap_df_r)

# Cleanup some of the data
snap_df['team'] = snap_df['team'].apply(lambda x: team_dic[x])
snap_df['side'] = snap_df['position'].apply(lambda x: snap_side_map[x])

snap_df = snap_df[ snap_df['side'] != 'DROP' ]

snap_df['position'] = snap_df['position'].apply(lambda x: snap_position_map[x])
snap_df['player_pfr'] = snap_df['player'].apply(lambda x: nflreadr.clean_player_names(x, convert_lastfirst=False)[0])
snap_df['join_string'] = snap_df['player_pfr'] + " " + snap_df['team'] + " " + snap_df['position']

snap_df

Note: nflreadr caches (i.e., stores a saved version) data by default.
If you expect different output try one of the following:
ℹ Restart your R Session or
ℹ Run `nflreadr::.clear_cache()`.
To disable this warning, run `options(nflreadr.verbose = FALSE)` or add it to your .Rprofile
This message is displayed once every 8 hours.


,game_id,pfr_game_id,season,game_type,week,player,pfr_player_id,position,team,opponent,offense_snaps,offense_pct,defense_snaps,defense_pct,st_snaps,st_pct,side,player_pfr,join_string
4,2023_01_ARI_WAS,202309100was,2023,REG,1,Sam Howell,HoweSa00,QB,WAS,ARI,71.0,1.00,0.0,0.0,0.0,0.00,OFF,Sam Howell,Sam Howell WAS QB
7,2023_01_ARI_WAS,202309100was,2023,REG,1,Terry McLaurin,McLaTe00,WR,WAS,ARI,63.0,0.89,0.0,0.0,0.0,0.00,OFF,Terry McLaurin,Terry McLaurin WAS WR
8,2023_01_ARI_WAS,202309100was,2023,REG,1,Jahan Dotson,DotsJa00,WR,WAS,ARI,62.0,0.87,0.0,0.0,0.0,0.00,OFF,Jahan Dotson,Jahan Dotson WAS WR
9,2023_01_ARI_WAS,202309100was,2023,REG,1,Logan Thomas,ThomLo00,TE,WAS,ARI,58.0,0.82,0.0,0.0,0.0,0.00,OFF,Logan Thomas,Logan Thomas WAS TE
10,2023_01_ARI_WAS,202309100was,2023,REG,1,Curtis Samuel,SamuCu00,WR,WAS,ARI,46.0,0.65,0.0,0.0,0.0,0.00,OFF,Curtis Samuel,Curtis Samuel WAS WR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79300,2021_22_LA_CIN,202202130cin,2021,SB,22,Christian Rozeboom,RozeCh00,LB,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Christian Rozeboom,Christian Rozeboom LAR LB
79301,2021_22_LA_CIN,202202130cin,2021,SB,22,Jake Gervase,GervJa00,S,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Jake Gervase,Jake Gervase LAR S
79302,2021_22_LA_CIN,202202130cin,2021,SB,22,Travin Howard,HowaTr00,LB,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Travin Howard,Travin Howard LAR LB
79303,2021_22_LA_CIN,202202130cin,2021,SB,22,Grant Haley,HaleGr00,CB,LAR,CIN,0.0,0.00,0.0,0.0,22.0,0.73,DEF,Grant Haley,Grant Haley LAR CB


In [8]:
# Dict to store data for each season
pos_df = {}

for s in seasons:
    # Scrape player positions from MFL and convert to pandas df
    mfl = ffscrapr.mfl_connect(season=s, league_id=mfl_id)
    
    pos_df_r = ffscrapr.mfl_players(mfl)
    
    with (ro.default_converter + pandas2ri.converter).context():
      pos_df[s] = ro.conversion.get_conversion().rpy2py(pos_df_r)

    # Cleanup some of the data
    pos_df[s]['season'] = s
    pos_df[s]['side'] = pos_df[s]['pos'].apply(lambda x: pos_side_map[x])

    pos_df[s] = pos_df[s][ pos_df[s]['side'] != 'DROP' ]
    
    pos_df[s]['player_mfl'] = pos_df[s]['player_name'].apply(lambda x: nflreadr.clean_player_names(x, convert_lastfirst=True)[0])
    pos_df[s]['join_string'] = pos_df[s]['player_mfl'] + " " + pos_df[s]['team'] + " " + pos_df[s]['pos']

# Merge seasons to single df
pos_df = pd.concat(pos_df, ignore_index=True)

# Sort by id and season for cleaner display
pos_df = pos_df.set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True]).reset_index()

# Save a copy of positions
pos_df[['player_id','player_mfl','season','team','side','pos','join_string']].to_csv(f'{save_label}_Positions.csv', index=False)

pos_df

,player_id,season,player_name,pos,age,team,status,draft_team,draft_year,draft_pick,...,nfl_id,twitter_username,weight,jersey,college,height,birthdate,side,player_mfl,join_string
0,0800,2022,"Weatherford, Sterling",S,NaN,IND,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,DEF,Sterling Weatherford,Sterling Weatherford IND S
1,0802,2022,"Krull, Lucas",TE,NaN,NOS,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,OFF,Lucas Krull,Lucas Krull NOS TE
2,0803,2022,"Vilain, Luiji",DE,NaN,MIN,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,DEF,Luiji Vilain,Luiji Vilain MIN DE
3,0806,2021,"Brown, Shakur",CB,NaN,PIT,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,DEF,Shakur Brown,Shakur Brown PIT CB
4,0808,2021,"Griffin, Olaijah",CB,NaN,BUF,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,DEF,Olaijah Griffin,Olaijah Griffin BUF CB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6902,9966,2021,"Jones, Reshad",S,36.0,FA,NA_character_,MIA,2010,32,...,reshadjones/496739,reshadjones9,215,20,Georgia,73,6629.0,DEF,Reshad Jones,Reshad Jones FA S
6903,9988,2021,"Brown, Antonio",WR,35.6,FA,NA_character_,PIT,2010,26,...,antoniobrown/2508061,AntonioBrown84,185,81,Central Michigan,70,6765.0,OFF,Antonio Brown,Antonio Brown FA WR
6904,9988,2022,"Brown, Antonio",WR,35.6,FA,NA_character_,PIT,2010,26,...,antoniobrown/2508061,AntonioBrown84,185,81,Central Michigan,70,6765.0,OFF,Antonio Brown,Antonio Brown FA WR
6905,9988,2023,"Brown, Antonio",WR,35.6,FA,NA_character_,PIT,2010,26,...,antoniobrown/2508061,AntonioBrown84,185,81,Central Michigan,70,6765.0,OFF,Antonio Brown,Antonio Brown FA WR


In [9]:
###
def GetClosestJoinString(other_df, this_js):
    # Return closest matching join string using fuzzy matching
    return fwprocess.extract(this_js, other_df['join_string'], scorer=fuzz.ratio)[0][0]

def GetClosestJoinScore(other_df, this_js):
    # Return score of closest match using fuzzy matching
    return fwprocess.extract(this_js, other_df['join_string'], scorer=fuzz.ratio)[0][1]

In [10]:
# Initialize merged df from snap data
df = snap_df[['game_id','season','week','pfr_player_id','player_pfr','team','position','side','join_string','offense_snaps','defense_snaps']].copy()

# Filter out weeks above max week
df = df[ df['week'] <= max_week ]

# Merge in positions using fuzzy string matching, considering each season and side of ball separately
df = df.set_index(['pfr_player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])

for s in seasons:
    for side in ['OFF','DEF']:
        print(s,side)

        # Constrain data to this season and side of ball
        tmp_snap_df = snap_df[ (snap_df['season'] == s) & (snap_df['side'] == side) ]
        tmp_snap_df = tmp_snap_df.set_index(['pfr_player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])
        tmp_snap_df = tmp_snap_df.groupby(['pfr_player_id','season']).tail(1).droplevel(2)

        tmp_pos_df = pos_df[ (pos_df['season'] == s) & (pos_df['side'] == side) ]

        # Fill the df with the closest matching join_string found by the fuzzy matching
        df.loc[ (df.index.get_level_values(1) == s) & (df['side'] == side), 'join_string'] = tmp_snap_df['join_string'].apply(lambda x: GetClosestJoinString(tmp_pos_df, x))

df = df.reset_index()

# With common join strings now in hand, we can merge in the pfl position and player data from MFL
df = df.set_index(['join_string','season']).sort_index(level=[0,1],ascending=[True,True])
df = df.join( pos_df.set_index(['join_string','season']).sort_index(level=[0,1],ascending=[True,True])[['pos','player_mfl']] )

df = df.reset_index()

# Rename the position columns
df = df.rename(columns={'position':'pfr_position'})
df = df.rename(columns={'pos':'position'})

# Find the per-game snap ranks for each player, assign qualifying players weight of 1, and break ties at the boundary with fractional weight
df = df.set_index(['game_id','team']).sort_index(level=[0,1],ascending=[True,True])

df['offense_rank'] = df.groupby(['game_id','team']).apply(lambda x: x['offense_snaps'].rank(method='first',ascending=False)).values  #Snap rank
df['offense_thresh'] = df.groupby(['game_id','team']).apply(lambda x: x['offense_snaps'][ x['offense_rank'] == num_off ].values[0])  #Snap boundary
df['offense_weight'] = ( (df['offense_rank'] <= num_off).values | (df['offense_snaps'] == df['offense_thresh']).values ).astype(float)  #Initial weights
thresh_inds = df['offense_snaps'] == df['offense_thresh']
df.loc[ thresh_inds, 'offense_weight' ] = df[thresh_inds].groupby(['game_id','team']).apply(lambda x: np.sum(x['offense_rank'] <= num_off) / len(x) )  #Normalize weights by breaking ties

df['defense_rank'] = df.groupby(['game_id','team']).apply(lambda x: x['defense_snaps'].rank(method='first',ascending=False)).values  #Snap rank
df['defense_thresh'] = df.groupby(['game_id','team']).apply(lambda x: x['defense_snaps'][ x['defense_rank'] == num_def ].values[0])  #Snap boundary
df['defense_weight'] = ( (df['defense_rank'] <= num_def).values | (df['defense_snaps'] == df['defense_thresh']).values ).astype(float)  #Initial weights
thresh_inds = df['defense_snaps'] == df['defense_thresh']
df.loc[ thresh_inds, 'defense_weight' ] = df[thresh_inds].groupby(['game_id','team']).apply(lambda x: np.sum(x['defense_rank'] <= num_def) / len(x) )  #Normalize weights by breaking ties

df = df.reset_index()

# Sort by season, week, game_id, and team for cleaner display
df = df.set_index(['season','week','game_id','team']).sort_index(level=[0,1,2,3],ascending=[True,True,True,True]).reset_index()

# Save a copy of the raw data
df.to_csv(f'{save_label}_Raw.csv', index=False)

df

2023 OFF
2023 DEF
2022 OFF
2022 DEF
2021 OFF
2021 DEF


,season,week,game_id,team,join_string,pfr_player_id,player_pfr,pfr_position,side,offense_snaps,defense_snaps,position,player_mfl,offense_rank,offense_thresh,offense_weight,defense_rank,defense_thresh,defense_weight
0,2021,1,2021_01_ARI_TEN,ARI,AJ Green ARI WR,GreeA.00,AJ Green,WR,OFF,55.0,0.0,WR,AJ Green,3.0,34.0,1.0,24.0,22.0,0.0
1,2021,1,2021_01_ARI_TEN,ARI,Antonio Hamilton ARI CB,HamiAn01,Antonio Hamilton,CB,DEF,0.0,3.0,CB,Antonio Hamilton,13.0,34.0,0.0,21.0,22.0,0.0
2,2021,1,2021_01_ARI_TEN,ARI,Budda Baker ARI S,BakeBu00,Budda Baker,S,DEF,0.0,61.0,S,Budda Baker,14.0,34.0,0.0,1.0,22.0,1.0
3,2021,1,2021_01_ARI_TEN,ARI,Byron Murphy ARI CB,MurpBy00,Byron Murphy,CB,DEF,0.0,58.0,CB,Byron Murphy,15.0,34.0,0.0,3.0,22.0,1.0
4,2021,1,2021_01_ARI_TEN,ARI,Chandler Jones ARI DE,JoneCh03,Chandler Jones,LB,DEF,0.0,48.0,DE,Chandler Jones,16.0,34.0,0.0,8.0,22.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54563,2023,17,2023_17_TEN_HOU,TEN,Tae Crowder FA LB,CrowTa00,Tae Crowder,LB,DEF,0.0,0.0,LB,Tae Crowder,35.0,23.0,0.0,33.0,28.0,0.0
54564,2023,17,2023_17_TEN_HOU,TEN,Terrell Edmunds TEN S,EdmuTe00,Terrell Edmunds,S,DEF,0.0,10.0,S,Terrell Edmunds,36.0,23.0,0.0,18.0,28.0,0.0
54565,2023,17,2023_17_TEN_HOU,TEN,Treylon Burks TEN WR,BurkTr00,Treylon Burks,WR,OFF,45.0,0.0,WR,Treylon Burks,2.0,23.0,1.0,34.0,28.0,0.0
54566,2023,17,2023_17_TEN_HOU,TEN,Tyjae Spears TEN RB,SpeaTy00,Tyjae Spears,RB,OFF,34.0,0.0,RB,Tyjae Spears,6.0,23.0,1.0,35.0,28.0,0.0


In [11]:
# Calculate the mean value for each season-team-position by summing over weights and dividing by number of games
df = df.set_index(['season','team','position']).sort_index(level=[0,1,2],ascending=[True,True,True])

result_off = df['offense_weight'].groupby(['season','team','position']).sum() / df.groupby(['season','team']).apply(lambda x: len(set(x['week'])))
result_off = result_off[ result_off >= 0.1 ]

result_def = df['defense_weight'].groupby(['season','team','position']).sum() / df.groupby(['season','team']).apply(lambda x: len(set(x['week'])))
result_def = result_def[ result_def >= 0.1 ]

df = df.reset_index()

# Pivot results out to columns to put into a nice tabular form
result_off = result_off.unstack(level=-1)
result_def = result_def.unstack(level=-1)

# Calculate values for DL and DB positions
result_def['DL'] = result_def['DT'] + result_def['DE']
result_def['DB'] = result_def['CB'] + result_def['S']

# Cleanup the table, listing the sorted values and season-teams for each position
result_dic = {}

for c in ['QB','RB','WR','TE']:
    tmp_df = result_off[c].reset_index()
    
    tmp_df[f'{c} Team'] = tmp_df['season'].map(str) + " " + tmp_df['team']
    tmp_df = tmp_df[[c, f'{c} Team']].rename(columns={c:f'{c} Value'})
    tmp_df = tmp_df.sort_values(f'{c} Value', ascending=False).reset_index(drop=True)

    result_dic[c] = tmp_df

for c in ['DT','DE','LB','CB','S','DL','DB']:
    tmp_df = result_def[c].reset_index()
    
    tmp_df[f'{c} Team'] = tmp_df['season'].map(str) + " " + tmp_df['team']
    tmp_df = tmp_df[[c, f'{c} Team']].rename(columns={c:f'{c} Value'})
    tmp_df = tmp_df.sort_values(f'{c} Value', ascending=False).reset_index(drop=True)

    result_dic[c] = tmp_df

# Save the table to a csv
result = pd.concat(result_dic, axis=1, ignore_index=False)
result.columns = result.columns.droplevel(0)

result.to_csv(f'{save_label}_Lineups.csv', index=False)

In [12]:
# Full results
result

,QB Value,QB Team,RB Value,RB Team,WR Value,WR Team,TE Value,TE Team,DT Value,DT Team,...,LB Value,LB Team,CB Value,CB Team,S Value,S Team,DL Value,DL Team,DB Value,DB Team
0,1.3750,2021 DEN,1.6875,2022 CLE,3.625000,2023 TBB,2.53125,2023 ATL,3.250000,2021 SEA,...,2.93750,2022 LAC,3.625000,2022 PHI,3.6250,2023 DAL,5.500000,2022 JAC,6.625000,2023 DAL
1,1.2500,2021 NOS,1.6250,2022 DAL,3.593750,2023 DAL,2.34375,2021 MIA,3.187500,2023 ATL,...,2.87500,2022 DAL,3.531250,2022 IND,3.2500,2021 NEP,5.406250,2023 CAR,5.875000,2022 DAL
2,1.1250,2023 NOS,1.6250,2022 LVR,3.562500,2023 BAL,2.09375,2021 LAC,3.156250,2023 CAR,...,2.81250,2022 ARI,3.500000,2022 CAR,3.0000,2022 BAL,5.375000,2022 MIA,5.687500,2022 NEP
3,1.1250,2021 HOU,1.6250,2023 CHI,3.531250,2022 CHI,2.09375,2022 TEN,3.125000,2021 WAS,...,2.75000,2023 BAL,3.437500,2023 TBB,3.0000,2023 MIN,5.343750,2021 JAC,5.687500,2021 SFO
4,1.0625,2023 ATL,1.6250,2023 PHI,3.500000,2021 ARI,2.09375,2022 BAL,3.125000,2022 ARI,...,2.71875,2022 NYJ,3.433333,2022 BUF,2.9375,2022 NEP,5.312500,2022 ATL,5.656250,2021 NOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1.0000,2022 JAC,1.0000,2021 PIT,2.562500,2021 DEN,1.15625,2023 MIA,1.562500,2021 DAL,...,1.71875,2023 CAR,2.312500,2022 ARI,1.6250,2022 IND,4.197917,2021 NOS,4.687500,2021 SEA
92,1.0000,2022 IND,1.0000,2021 NEP,2.520833,2022 ATL,1.12500,2022 LAR,1.531250,2021 SFO,...,1.68750,2023 LAR,2.312500,2022 SFO,1.5625,2021 HOU,4.187500,2021 DAL,4.562500,2022 ATL
93,1.0000,2022 HOU,1.0000,2023 DAL,2.406250,2021 MIA,1.12500,2021 ARI,1.510417,2021 NOS,...,1.62500,2021 LAR,2.218750,2023 MIN,1.5625,2023 TBB,4.187500,2022 LAC,4.541667,2022 MIA
94,1.0000,2022 GBP,1.0000,2023 LAC,2.406250,2022 BAL,1.06250,2023 BAL,1.437500,2023 DAL,...,1.59375,2023 MIN,2.187500,2022 BAL,1.4375,2021 BAL,4.062500,2023 DAL,4.437500,2022 JAC


In [13]:
# Team 2
result.iloc[1]

QB Value        1.25
QB Team     2021 NOS
RB Value       1.625
RB Team     2022 DAL
WR Value     3.59375
WR Team     2023 DAL
TE Value     2.34375
TE Team     2021 MIA
DT Value      3.1875
DT Team     2023 ATL
DE Value    3.114583
DE Team     2022 KCC
LB Value       2.875
LB Team     2022 DAL
CB Value     3.53125
CB Team     2022 IND
S Value         3.25
S Team      2021 NEP
DL Value     5.40625
DL Team     2023 CAR
DB Value       5.875
DB Team     2022 DAL
Name: 1, dtype: object

In [14]:
# Team 48 (half of median)
result.iloc[47]

QB Value         1.0
QB Team     2023 JAC
RB Value        1.25
RB Team     2021 GBP
WR Value      3.0625
WR Team     2023 JAC
TE Value    1.635417
TE Team     2022 WAS
DT Value      2.5625
DT Team     2022 PIT
DE Value    2.291667
DE Team     2022 PHI
LB Value     2.03125
LB Team     2023 NYG
CB Value      2.9375
CB Team     2023 CLE
S Value        2.125
S Team      2022 MIA
DL Value     4.90625
DL Team     2023 SEA
DB Value      5.0625
DB Team     2022 PIT
Name: 47, dtype: object

In [15]:
# Team 49 (half of median)
result.iloc[48]

QB Value         1.0
QB Team     2022 SEA
RB Value        1.25
RB Team     2021 MIA
WR Value      3.0625
WR Team     2023 PIT
TE Value       1.625
TE Team     2023 SEA
DT Value      2.5625
DT Team     2021 TBB
DE Value    2.291667
DE Team     2021 NYG
LB Value         2.0
LB Team     2023 ATL
CB Value     2.90625
CB Team     2022 KCC
S Value        2.125
S Team      2021 CLE
DL Value       4.875
DL Team     2022 NYG
DB Value      5.0625
DB Team     2023 LAC
Name: 48, dtype: object

In [16]:
# Team 95
result.iloc[94]

QB Value         1.0
QB Team     2022 GBP
RB Value         1.0
RB Team     2023 LAC
WR Value     2.40625
WR Team     2022 BAL
TE Value      1.0625
TE Team     2023 BAL
DT Value      1.4375
DT Team     2023 DAL
DE Value     1.59375
DE Team     2022 DET
LB Value     1.59375
LB Team     2023 MIN
CB Value      2.1875
CB Team     2022 BAL
S Value       1.4375
S Team      2021 BAL
DL Value      4.0625
DL Team     2023 DAL
DB Value      4.4375
DB Team     2022 JAC
Name: 94, dtype: object

In [17]:
# Examine instances where player names don't match
print(len(df[ (df['player_pfr'] != df['player_mfl']) & (df['offense_weight'] + df['defense_weight'] > 0) ]))

97


In [18]:
df[ (df['player_pfr'] != df['player_mfl']) & (df['offense_weight'] + df['defense_weight'] > 0) ].head(60)

,season,team,position,week,game_id,join_string,pfr_player_id,player_pfr,pfr_position,side,offense_snaps,defense_snaps,player_mfl,offense_rank,offense_thresh,offense_weight,defense_rank,defense_thresh,defense_weight
10917,2021,MIA,DE,1,2021_01_MIA_NE,Sam Eguavoen MIA DE,EguaSa00,Samuel Eguavoen,LB,DEF,0.0,36.0,Sam Eguavoen,35.0,21.0,0.0,12.0,36.0,1.0
10948,2021,MIA,DE,7,2021_07_ATL_MIA,Sam Eguavoen MIA DE,EguaSa00,Samuel Eguavoen,LB,DEF,0.0,53.0,Sam Eguavoen,33.0,30.0,0.0,4.0,31.0,1.0
17791,2021,WAS,DT,2,2021_02_NYG_WAS,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,43.0,Matt Ioannidis,31.0,31.0,0.0,11.0,39.0,1.0
17801,2021,WAS,DT,4,2021_04_WAS_ATL,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,44.0,Matt Ioannidis,31.0,27.0,0.0,11.0,33.0,1.0
17806,2021,WAS,DT,5,2021_05_NO_WAS,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,40.0,Matt Ioannidis,31.0,34.0,0.0,10.0,22.0,1.0
17816,2021,WAS,DT,7,2021_07_WAS_GB,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,27.0,Matt Ioannidis,32.0,30.0,0.0,12.0,27.0,1.0
17821,2021,WAS,DT,8,2021_08_WAS_DEN,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,29.0,Matt Ioannidis,31.0,23.0,0.0,12.0,29.0,1.0
17826,2021,WAS,DT,10,2021_10_TB_WAS,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,25.0,Matt Ioannidis,32.0,33.0,0.0,12.0,25.0,1.0
17832,2021,WAS,DT,11,2021_11_WAS_CAR,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,24.0,Matt Ioannidis,32.0,30.0,0.0,12.0,24.0,1.0
17844,2021,WAS,DT,13,2021_13_WAS_LV,Matt Ioannidis WAS DT,IoanMa01,Matthew Ioannidis,DT,DEF,0.0,33.0,Matt Ioannidis,32.0,34.0,0.0,10.0,31.0,1.0


In [19]:
df[ (df['player_pfr'] != df['player_mfl']) & (df['offense_weight'] + df['defense_weight'] > 0) ].tail(60)

,season,team,position,week,game_id,join_string,pfr_player_id,player_pfr,pfr_position,side,offense_snaps,defense_snaps,player_mfl,offense_rank,offense_thresh,offense_weight,defense_rank,defense_thresh,defense_weight
36006,2022,WAS,LB,10,2022_10_WAS_PHI,Jon Bostic WAS LB,BostJo00,Jonathan Bostic,LB,DEF,0.0,43.0,Jon Bostic,28.0,40.0,0.0,8.0,22.0,1.0
36011,2022,WAS,LB,11,2022_11_WAS_HOU,Jon Bostic WAS LB,BostJo00,Jonathan Bostic,LB,DEF,0.0,42.0,Jon Bostic,27.0,35.0,0.0,9.0,27.0,1.0
36016,2022,WAS,LB,12,2022_12_ATL_WAS,Jon Bostic WAS LB,BostJo00,Jonathan Bostic,LB,DEF,0.0,27.0,Jon Bostic,27.0,30.0,0.0,12.0,27.0,1.0
36021,2022,WAS,LB,13,2022_13_WAS_NYG,Jon Bostic WAS LB,BostJo00,Jonathan Bostic,LB,DEF,0.0,39.0,Jon Bostic,26.0,42.0,0.0,8.0,35.0,1.0
36026,2022,WAS,LB,15,2022_15_NYG_WAS,Jon Bostic WAS LB,BostJo00,Jonathan Bostic,LB,DEF,0.0,45.0,Jon Bostic,26.0,36.0,0.0,10.0,25.0,1.0
36898,2023,ATL,CB,1,2023_01_CAR_ATL,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,0.0,72.0,Dee Alford,22.0,26.0,0.0,6.0,34.0,1.0
36902,2023,ATL,CB,2,2023_02_GB_ATL,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,0.0,24.0,Dee Alford,23.0,38.0,0.0,12.0,24.0,1.0
36906,2023,ATL,CB,3,2023_03_ATL_DET,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,0.0,64.0,Dee Alford,18.0,20.0,0.0,5.0,19.0,1.0
36911,2023,ATL,CB,4,2023_04_ATL_JAX,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,0.0,40.0,Dee Alford,21.0,19.0,0.0,10.0,28.0,1.0
36917,2023,ATL,CB,5,2023_05_HOU_ATL,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,0.0,36.0,Dee Alford,22.0,35.0,0.0,10.0,28.0,1.0
